In [ ]:
import pycinema
import PIL
import numpy as np
from matplotlib import pyplot

## Create the instance of the filter
mlfilter = pycinema.ImageGeneratorCNN()

## Input trained model file
mlfilter.inputs.Model.set('testing/gold/artifact/MLModels/Model_MLImageGenerator_Isabel_timevarying_2000.pth',False)
mlfilter.inputs.Device.set('cpu',False)
## Phi,Theta, Timestep value for which the image will be generated
mlfilter.inputs.Params.set([[-10,5.0,3]],False)
## number of input parameter in training set, in this case, 2 (phi and theta)
mlfilter.inputs.VP.set(3,False)
## Internal parameter for CNN, should be the same as was used during training
mlfilter.inputs.VPO.set(256,False)
## Internal parameter for CNN, should be the same as was used during training
mlfilter.inputs.Channel.set(8,False)

# Add annotation
annotation = pycinema.Annotation()
annotation.inputs.Images.set(mlfilter.outputs.Images,False)
annotation.inputs.Color.set((255,0,0))
annotation.inputs.Ignore.set(['FILE', 'Time', 'Phi', 'Theta'])
annotation.inputs.Size.set(10)
annotation.inputs.XY.set((5,5))

# update the pipeline
mlfilter.update()

# show the generated images
images = annotation.outputs.Images.get();
for i in range(len(images)):
    image = images[i]
    pyplot.imshow(image.channel['RGBA'])
    pyplot.axis('off')
    pyplot.show()